In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Dropout, \
    Conv2D, MaxPooling2D, AveragePooling2D
from keras.metrics import AUC, Recall
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from _common import NUM_LABEL_COLS, RANDOM_SEED
from _tenor_lmsc import TEST_SIZE, VALIDATION_SIZE, \
    BANDS, TIME_SLICES, master, lmss, data, target
# Note that records are read from pickle, selected, filtered,
# and scaled with `sklearn.preprocessing.StandardScaler` 
# in `_tenor_lmsc.py`.

In [5]:
x_train, x_test, y_train, y_test = \
     train_test_split(data, target,
                      test_size=TEST_SIZE,
                      random_state=RANDOM_SEED)

In [6]:
# Reshape for input to 2D convolutional layer
x_train = x_train.reshape(x_train.shape[0], BANDS, TIME_SLICES, 1)
x_test = x_test.reshape(x_test.shape[0], BANDS, TIME_SLICES, 1)

In [9]:
dependencies = {
    'recall': Recall
}

fn = './keras_models/tenor_cnn/tenor_cnn_F.h5'
model = load_model(fn, custom_objects=dependencies, compile=False)

In [11]:
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=[Recall(name='recall'), 'accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
# auc, 
loss, recall, accuracy = model.evaluate(x_test, y_test)
# print('AUC: %.2f' % (auc * 100))
print('Recall: %.2f' % (recall * 100))
print('Accuracy: %.2f' % (accuracy * 100))
# @ 3 Epochs
# Recall: 83.33
# Accuracy: 87.72
# @ 6 Epochs
# Recall: 94.61
# Accuracy: 93.02
# @ 12 Epochs
# Recall: 95.59
# Accuracy: 91.96


1132/1132 [==============================] - 8s 7ms/step
Recall: 95.59
Accuracy: 91.96
